In [7]:
import math
from datetime import datetime, timedelta

import numpy as np
import optuna
import pandas as pd
#import xgboost as xgb
#from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow

In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [9]:
def get_or_create_experiment(experiment_name):
  """
  Retrieve the ID of an existing MLflow experiment or create a new one if it doesn't exist.

  This function checks if an experiment with the given name exists within MLflow.
  If it does, the function returns its ID. If not, it creates a new experiment
  with the provided name and returns its ID.

  Parameters:
  - experiment_name (str): Name of the MLflow experiment.

  Returns:
  - str: ID of the existing or newly created MLflow experiment.
  """

  if experiment := mlflow.get_experiment_by_name(experiment_name):
      return experiment.experiment_id
  else:
      return mlflow.create_experiment(experiment_name)

In [13]:
# Create an experiment for our hyperparameter tuning runs
experiment_id = get_or_create_experiment("First Experiment")

In [14]:
experiment_id

'3'

In [15]:
# Set the current active MLflow experiment
mlflow.set_experiment(experiment_id=experiment_id)

# Load and preprocess the data
data = pd.read_csv("WineQt.csv")

# Rename columns to replace spaces with underscores
data.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

# Create a binary target variable for high quality wines
high_quality = (data.quality >= 7).astype(int)
data['high_quality'] = high_quality

#
X = data.drop(["quality", "high_quality", "Id"], axis=1)
y = data["high_quality"]

# Split out the data
X_train, y_train, X_test, y_test = train_test_split(X, y, train_size=0.7, random_state=123)



In [16]:
# override Optuna's default logging to ERROR only
optuna.logging.set_verbosity(optuna.logging.ERROR)

# define a logging callback that will report on only new challenger parameter configurations if a
# trial has usurped the state of 'best conditions'

def champion_callback(study, frozen_trial):
  """
  Logging callback that will report when a new trial iteration improves upon existing
  best trial values.

  Note: This callback is not intended for use in distributed computing systems such as Spark
  or Ray due to the micro-batch iterative implementation for distributing trials to a cluster's
  workers or agents.
  The race conditions with file system state management for distributed trials will render
  inconsistent values with this callback.
  """

  winner = study.user_attrs.get("winner", None)

  if study.best_value and winner != study.best_value:
      study.set_user_attr("winner", study.best_value)
      if winner:
          improvement_percent = (abs(winner - study.best_value) / study.best_value) * 100
          print(
              f"Trial {frozen_trial.number} achieved value: {frozen_trial.value} with "
              f"{improvement_percent: .4f}% improvement"
          )
      else:
          print(f"Initial trial {frozen_trial.number} achieved value: {frozen_trial.value}")

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import mlflow

def objective(trial):
    with mlflow.start_run(nested=True):

        # --- Hyperparameters to tune ---
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 500),
            "max_depth": trial.suggest_int("max_depth", 5, 30),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 5, 50),
            "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
            "class_weight": "balanced",
            "n_jobs": 1,
            "random_state": 123,
        }

        # --- Train model ---
        model = RandomForestClassifier(**params)
        model.fit(X_train, y_train)

        # --- Evaluate ---
        proba = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, proba)

        # --- Log to MLflow ---
        mlflow.log_params(params)
        mlflow.log_metric("test_auc", auc)

    # Optuna minimizes by default → return negative
    return -auc


In [18]:
run_name = "first_attempt"

In [21]:
import mlflow.sklearn

# Initiate the parent run and call the hyperparameter tuning child run logic
with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True):
  # Initialize the Optuna study
  study = optuna.create_study(direction="minimize")

  # Execute the hyperparameter optimization trials.
  # Note the addition of the `champion_callback` inclusion to control our logging
  study.optimize(objective, n_trials=10, callbacks=[champion_callback])

  mlflow.log_params(study.best_params)
  mlflow.log_metric("best_auc", study.best_value)

  # Log tags
  mlflow.set_tags(
      tags={
          "project": "Wine Quality Project",
          "optimizer_engine": "optuna",
          "model_family": "randomforest",
          "feature_set_version": 1,
      }
  )

  # Log a fit model instance
  model = RandomForestClassifier(**study.best_params)
  model.fit(X_train, y_train)

  artifact_path = "model"

  mlflow.sklearn.log_model(
      sk_model=model,
      name=artifact_path,
      input_example=X_train.iloc[[0]],
      model_format="ubj",
      metadata={"model_data_version": 1},
  )

  # Get the logged model uri so that we can load it from the artifact store
  model_uri = mlflow.get_artifact_uri(artifact_path)

🏃 View run enchanting-pug-536 at: http://127.0.0.1:5000/#/experiments/3/runs/81b182b972c5473390fe797ad1995503
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
🏃 View run first_attempt at: http://127.0.0.1:5000/#/experiments/3/runs/b5a4766a9ede49838562678ef5eb0aae
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3


ValueError: Found input variables with inconsistent numbers of samples: [800, 343]

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
957,8.2,0.44,0.24,2.3,0.063,10.0,28.0,0.99613,3.25,0.53,10.2


In [8]:
data = pd.read_csv("WineQt.csv")

data.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

high_quality = (data.quality >= 7).astype(int)
data['high_quality'] = high_quality

data.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,Id,high_quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4,0


In [10]:
from sklearn.model_selection import train_test_split

X = data.drop(["quality", "high_quality", "Id"], axis=1)
y = data["high_quality"]

# Split out the training data
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.6, random_state=123)

# Split the remaining data equally into validation and test
X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=123)

0    0
1    0
2    0
3    0
4    0
Name: high_quality, dtype: int64

In [ ]:
def plot_feature_importance(model, booster):
  """
  Plots feature importance for an XGBoost model.

  Args:
  - model: A trained XGBoost model

  Returns:
  - fig: The matplotlib figure object
  """
  fig, ax = plt.subplots(figsize=(10, 8))
  importance_type = "weight" if booster == "gblinear" else "gain"
  xgb.plot_importance(
      model,
      importance_type=importance_type,
      ax=ax,
      title=f"Feature Importance based on {importance_type}",
  )
  plt.tight_layout()
  plt.close(fig)

  return fig